In [2]:
!where python

C:\Users\tinkie\anaconda3\python.exe
C:\Users\tinkie\AppData\Local\Microsoft\WindowsApps\python.exe


In [3]:
import pandas as pd
import numpy as np

In [4]:
def load_data(split_name='train', columns=['text', 'stars']):
    try:
        print(f"select [{', '.join(columns)}] columns from the {split_name} split")
        df = pd.read_csv(f'{split_name}.csv')
        df = df.loc[:,columns]
        print("succeed!")
        return df
    except:
        print("Failed, then try to ")
        print(f"select all columns from the {split_name} split")
        df = pd.read_csv(f'{split_name}.csv')
        return df

In [5]:
matrix = load_data('matrix', "full")
subset = matrix.iloc[509:, 0:]
movies = load_data('movies', ['movieId', 'title'])

select [f, u, l, l] columns from the matrix split
Failed, then try to 
select all columns from the matrix split
select [movieId, title] columns from the movies split
succeed!


In [ ]:
def cos_sim(userA, userB):
    userArt = subset.loc[subset['UserId'] == userA].to_numpy().flatten()[1:]
    userBrt = subset.loc[subset['UserId'] == userB].to_numpy().flatten()[1:]
    mask = ~(np.isnan(userArt) | np.isnan(userBrt))
    if mask.sum() < 4:
         return -2
    return np.dot(userArt[mask], userBrt[mask]) / (np.sqrt(np.dot(userArt[mask], userArt[mask])) * np.sqrt(np.dot(userBrt[mask], userBrt[mask])))

In [63]:
def find_most_sim_user(cur_user):
    max_sim = -2
    max_user = -1
    for user in subset['UserId'].to_numpy():
        if user == cur_user:
            continue
        sim = cos_sim(cur_user, user)
        if sim > max_sim:
            max_sim = sim
            max_user = user
    return max_user

In [64]:
find_most_sim_user(612)

611

In [65]:
def find_new_movie(cur_user , sim_user):
    userArt = subset.loc[subset['UserId'] == cur_user].to_numpy().flatten()[1:]
    userBrt = subset.loc[subset['UserId'] == sim_user].to_numpy().flatten()[1:]
    mask = np.isnan(userArt) & ~(np.isnan(userBrt))
    movieIds = np.argwhere(mask).flatten()+1
    np.random.shuffle(movieIds)
    return movies['title'] [movies['movieId'].isin(movieIds[:10])].to_numpy()

In [ ]:
movieIds = find_new_movie(605,599)

movieIds[:10]

In [ ]:
movies = load_data('movies', ['movieId', 'title'])
movies.head()

In [83]:
find_new_movie(612,611)

array(['Grumpier Old Men (1995)', 'Father of the Bride Part II (1995)',
       'Balto (1995)', 'Four Rooms (1995)',
       'Ace Ventura: When Nature Calls (1995)', 'Money Train (1995)',
       'To Die For (1995)', 'Pocahontas (1995)'], dtype=object)

In [57]:
def gen_suggestion(cur_user):
    sim_user = find_most_sim_user(cur_user)
    return find_new_movie(cur_user, sim_user)

In [6]:
gen_suggestion(612)

NameError: name 'gen_suggestion' is not defined